# Plot Cote d'Ivoire Monthly Rainfall Data #3

Maps with a mask for Cote d'Ivoire

In [ ]:
import glob
import numpy as np
import xarray as xr
import hvplot.xarray
import hvplot.pandas
#import geoviews.feature as gf
import panel as pn
import colorcet as cc

#set the backend and the renderer
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')
br = hv.renderer('bokeh')

In [ ]:
# Set the data directory 
nc_dir = '../data/rainfall/'

## Load Data into Xarray Dataset

Load multiple netCDF-4 files using Xarray. Call compute() to load the data into memory since xr.open_mfdataset() uses Dask to lazily load the files resulting in chunked data. Calling compute() to put all the data into memory.  

In [ ]:
# 1. Load the netCDF-4 file into an Xarray dataset
# 2. Print the Dataset header
dpr_all_ds = xr.open_mfdataset(nc_dir + '*.nc4').compute()
dpr_all_ds

## Mask Dataset with Shapefile

https://gis.stackexchange.com/questions/354782/masking-netcdf-time-series-data-from-shapefile-using-python/354798#354798

In [ ]:
import rioxarray
import geopandas
from shapely.geometry import mapping

In [ ]:
dpr_all_ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
dpr_all_ds.rio.write_crs("WGS 84", inplace=True)
shapefile_path = '../data/shapefiles/civ_admbnda_adm0_cntig_20180706/civ_admbnda_adm0_cntig_20180706.shp'
Cote_DIvoire_Shape = geopandas.read_file(shapefile_path, crs="WGS 84")
Cote_DIvoire_Shape

In [ ]:
dpr_all_clipped_ds = dpr_all_ds.rio.clip(Cote_DIvoire_Shape.geometry.apply(mapping), Cote_DIvoire_Shape.crs, drop=False)
dpr_all_clipped_ds

## Annual Mean Rainfall Map

In [ ]:
monthly_mean_rainfall_plot = \
dpr_all_clipped_ds.all_rainfall.mean(dim='time').\
    hvplot.quadmesh(x='lon', y='lat',                                    # Call hvplot's QuadMesh mapping function and set the x and y coordinate variables
                    xlim=(-9.0,-2.0), ylim=(4.0, 11.0),                  # Set the limits of the x and y dimensions 
                    rasterize=True,                                      # Make it plot faster
                    cmap='rainbow', clim=(0,8), clabel='mm/day',         # Set the colormap and it's bounds
                    frame_width=500, frame_height=400,                  # Plot's width is 300 pixels
                    title='Annual Rainfall Mean',
                    xlabel='Longitude', ylabel='Latitude',
                    fontscale=1.5,
                    )

monthly_mean_rainfall_plot

## Monthly Mean Rainfall Map

In [ ]:
# Monthly Mean Rainfall

sf_monthly_mean_rainfall_plot = \
dpr_all_clipped_ds.sf_rainfall.groupby('time.month').mean().\
    hvplot.quadmesh(x='lon', y='lat',                                    # Call hvplot's QuadMesh mapping function and set the x and y coordinate variables
                    xlim=(-9.0,-2.0), ylim=(4.0, 11.0),                  # Set the limits of the x and y dimensions 
                    rasterize=True,                                      # Make it plot faster
                    cmap='rainbow', clim=(0,20),                         # Set the colormap and it's bounds
                    frame_width=300, frame_height=250,                   # Plot's width is 300 pixels
                    title='Stratiform Rainfall',
                    )

# Convective Non-shallow Plot
conv_non_sh_monthly_mean_rainfall_plot = \
dpr_all_clipped_ds.conv_non_sh_rainfall.groupby('time.month').mean().\
    hvplot.quadmesh(x='lon', y='lat',
                    xlim=(-9.0,-2.0), ylim=(4.0, 11.0),
                    rasterize=True, 
                    cmap='rainbow', clim=(0,20),
                    frame_width=300, frame_height=250,
                    title='Convective Non-shallow Rainfall')

# Convective Shallow Plot
conv_sh_monthly_mean_rainfall_plot = \
dpr_all_clipped_ds.conv_sh_rainfall.groupby('time.month').mean().\
    hvplot.quadmesh(x='lon', y='lat',
                    xlim=(-9.0,-2.0), ylim=(4.0, 11.0),
                    rasterize=True, 
                    cmap='rainbow', clim=(0,3),
                    frame_width=300, frame_height=250,
                    title='Convective Shallow Rainfall')

# Set month slider to appear at bottom of the plot
hv.output(widget_location='bottom')

# Plot all 3 in a single row
sf_monthly_mean_rainfall_plot + conv_non_sh_monthly_mean_rainfall_plot + conv_sh_monthly_mean_rainfall_plot

## Monthly Rainfall Anomaly from Annual Climatological Mean

https://xarray.pydata.org/en/v2022.03.0/examples/weather-data.html

https://colorcet.holoviz.org/user_guide/index.html

In [ ]:
# Create Climatological Mean Rainfall 
climo_mean_rainfall_da = dpr_all_clipped_ds.all_rainfall.mean('time')
climo_mean_rainfall_da

# Create Monthly Mean Rainfall
monthly_mean_rainfall_da = dpr_all_clipped_ds.all_rainfall.groupby('time.month').mean()
monthly_mean_rainfall_da

# Fine the monthly difference from climatological mean
monthly_mean_rainfall_anom_da = monthly_mean_rainfall_da - climo_mean_rainfall_da

# Select colormap, we'll reverse it so green is positive anomaly
cmap = cc.CET_D3[::-1]

# Plot monthly anomaly
monthly_mean_rainfall_anom_da.hvplot.quadmesh(x='lon', y='lat', 
                                              xlim=(-9.0,-2.0), ylim=(4.0, 11.0),
                                              cmap=cmap, clim=(-15,15), clabel='mm/day',
                                              xlabel='Longitude', ylabel='Latitude',
                                              frame_width=500, frame_height=400,
                                              title='Monthly Rainfall Anomaly', 
                                              fontscale=1.5)